In [1]:
%pip install ultralytics
%pip install super-gradients
%pip install supervision
%pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.4/715.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 8.7 MB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 2.8 MB/s eta 0:00:00


In [2]:
from ultralytics import NAS

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from super_gradients.training import models
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"

best_model = models.get(
    'yolo_nas_l',
    num_classes=1,
    checkpoint_path="/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/average_model.pth"
).to(DEVICE)


[2024-02-20 18:42:43] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


[2024-02-20 18:42:44] WARNING - __init__.py - Failed to import pytorch_quantization
[2024-02-20 18:42:52] WARNING - calibrator.py - Failed to import pytorch_quantization
[2024-02-20 18:42:52] WARNING - export.py - Failed to import pytorch_quantization
[2024-02-20 18:42:52] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2024-02-20 18:42:52] WARNING - env_sanity_check.py - Failed to verify installed packages: boto3 required but not found
[2024-02-20 18:42:52] WARNING - env_sanity_check.py - Failed to verify installed packages: deprecated required but not found
[2024-02-20 18:42:52] WARNING - env_sanity_check.py - Failed to verify installed packages: coverage required but not found
[2024-02-20 18:42:52] WARNING - env_sanity_check.py - Failed to verify installed packages: sphinx-rtd-theme required but not found
[2024-02-20 18:42:52] WARNING - env_sanity_check.py - Failed to verify installed packages: torchmetrics required but not found
[2024-02-20 18:42:

In [ ]:
import supervision as sv
import ultralytics

def get_inference(path_project):
  ds = sv.DetectionDataset.from_yolo(
      images_directory_path=f"{path_project}/images",
      annotations_directory_path=f"{path_project}/labels",
      data_yaml_path=f"/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/data.yaml",
      force_masks=False
  )

  CONFIDENCE_TRESHOLD = 0.5
  predictions = {}
  for image_name, image in ds.images.items():
    result =best_model.predict(image, conf=CONFIDENCE_TRESHOLD)
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int),
    )
    predictions[image_name] = detections

    nome_imagem = image_name.split('/')[7] + ".txt"
    with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/inferencias/{nome_imagem}","w") as file:
      if(len(detections.xyxy) > 0):
        file.write(str(detections.xyxy[-1]))
      else:
        file.write('[]')

get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/test")
get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train")
get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid")

In [5]:
import supervision as sv
import ultralytics
box_annotator = sv.BoxAnnotator()

images = []
titles = []

detections_list = []
prediction_list = []

with open("/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/detections.txt",'w') as file:
  file.write('')
with open("/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/predictions.txt",'w') as file:
  file.write('')

def get_ioudata(path_project):
  ds = sv.DetectionDataset.from_yolo(
      images_directory_path=f"{path_project}/images",
      annotations_directory_path=f"{path_project}/labels",
      data_yaml_path=f"/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3-allinone/data.yaml",
      force_masks=False
  )
  CONFIDENCE_TRESHOLD = 0.5
  predictions = {}
  for image_name, image in ds.images.items():
    result = best_model.predict(image, conf=CONFIDENCE_TRESHOLD)
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int)
    )
    predictions[image_name] = detections

  keys = list(ds.images.keys())
  for key in keys:
      # Para anotações
      annotation_detections = ds.annotations[key]
      for detection in annotation_detections:
          detections_list.append(detection[0])
          with open("/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/detections.txt",'a') as file:
            file.write(str(detection)+'\n')

      # Para previsões
      if len(predictions[key].xyxy) > 0:
        predictions[key].xyxy = predictions[key].xyxy[-1]
      prediction_detections = predictions[key]
      prediction_list.append(prediction_detections)
      with open("/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/predictions.txt",'a') as file:
        file.write(str(prediction_detections )+ '\n')

get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3-allinone/data")


/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
[2024-02-20 18:45:37] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-02-20 18:45:41] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-02-20 18:45:45] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-02-20 18:45:49] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-02-20 18:45:53] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-02-20 18

In [6]:
import numpy as np

with open("/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/Iou.txt",'w') as file:
      file.write('')

def calculate_iou(box1, box2):
    """
    Calcula a interseção sobre união (IoU) de dois bounding boxes no formato xyxy.

    Args:
    - box1 (list): Lista contendo as coordenadas [x1, y1, x2, y2] do primeiro bounding box.
    - box2 (list): Lista contendo as coordenadas [x3, y3, x4, y4] do segundo bounding box.

    Returns:
    - float: Valor da IoU entre os dois bounding boxes.
    """
    # Calcular a interseção
    x_intersection = max(0, min(box1[2], box2[2]) - max(box1[0], box2[0]))
    y_intersection = max(0, min(box1[3], box2[3]) - max(box1[1], box2[1]))
    area_intersection = x_intersection * y_intersection

    # Calcular a união
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    area_union = area_box1 + area_box2 - area_intersection

    # Calcular a IoU
    iou = area_intersection / max(area_union, 1e-10)  # Evitar divisão por zero

    return iou

for i in range(len(detections_list)):
  try:
    iou_score = calculate_iou(detections_list[i], prediction_list[i].xyxy)
  except Exception as e:
    print("fim das previsões", i, e)
    iou_score = 0.0
  finally:
    with open("/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/Iou.txt",'a') as file:
      file.write(str(iou_score)+ '\n')



fim das previsões 3 index 2 is out of bounds for axis 0 with size 0
fim das previsões 70 index 2 is out of bounds for axis 0 with size 0
fim das previsões 191 index 2 is out of bounds for axis 0 with size 0


In [66]:
keys[49]

NameError: name 'keys' is not defined

214.43999
263.845
75.17
106.55
174.83
184.615
202.31
143.79001
327.69
71.335
186.67
195.385
118.46
66.07
291.535
67.855
263.08
333.335
366.66998
294.62
69.415
41.379997
18.210001
163.175
122.41501
80.340004
52.76
271.54
141.38
66.67
98.275
44.14
93.795
155.72
81.72
64.48
232.305
178.46
212.31
131.385
220.77
152.755
70.35
260.0
144.615
294.615
263.84
27.585001
171.535
112.505
36.07
204.61
124.48
252.31
154.7
122.630005
135.51001
122.725
32.670002
133.45
102.065
74.135
92.409996
54.285
81.034996
33.155003
33.7
56.55
302.31
65.770004
135.385
143.99501
155.29001
70.0
45.86
62.309998
115.51499
147.93
120.77
108.840004
74.14
120.0
235.385
285.38
98.125
271.54
170.765
112.07
97.369995
84.485
279.23
120.70999
105.200005
32.415
294.615
101.384995
214.005
112.305
26.210001
61.030003
113.85
122.85999
116.880005
103.795
120.77
114.615
150.775
108.62
110.350006
208.46
254.705
155.39
68.62
113.795
95.86
225.38
62.07
149.995
83.0
102.065
123.075
199.995
104.284996
186.155
210.765
141.54
118.62
294.16